In [9]:
import pandas as pd 
from xgboost import XGBClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler
from scipy import stats
from matplotlib.legend_handler import HandlerLine2D
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [10]:
def read_input():
    data = pd.read_csv('train.csv', header=0) 
    data = data.sample(frac=1).reset_index(drop=True)
    np_data= data.values
    labels = np_data[:,1].astype('int')
    train= np_data[:,2:].astype(np.float32)
    print("Train Shape", train.shape)
    test = pd.read_csv("test.csv", header=0).values 
    ids = test[:,0]
    test= test[:,1:].astype(np.float32)
    print("Test Shape", test.shape)
    return train,labels, test, ids

def remove_outlier(train, labels):
    z = np.abs(stats.zscore(train))
    threshold = 3
    x,y=np.where(z > 3)
    lst= np.unique(x)
    all_indices = [i for i in range(0, train.shape[0])]
    non_outlier_indices = list(set(all_indices)-set(lst))
    train2 = train[non_outlier_indices,:]
    labels2 = labels[non_outlier_indices]
    return train2, labels2

def normalize(train):
    scaler = StandardScaler()
    scaler.fit(train)

    return scaler
def min_max_normalize(train):
    scaler = MinMaxScaler()
    scaler.fit(train)
    return scaler

#Append few extra features/columns
def append_features(train):
    min_col = np.min(train, axis=1).reshape((-1,1))
    max_col = np.max(train, axis=1).reshape((-1,1))
    mean_col = np.mean(train, axis=1).reshape((-1,1))
    std_col = np.std(train, axis=1).reshape((-1,1))
    sum_col = np.sum(train, axis=1).reshape((-1,1))
    med_col = np.median(train, axis=1).reshape((-1,1))

    new_train = train.copy()
    
    new_train=np.column_stack((new_train, min_col))
    new_train=np.column_stack((new_train, max_col))
    new_train=np.column_stack((new_train, mean_col))
    new_train=np.column_stack((new_train, std_col))
    new_train=np.column_stack((new_train, sum_col))
    new_train=np.column_stack((new_train, med_col))
    
   # lst= [0.01,0.05,0.10,0.25,0.50,0.60, 0.70, 0.80, 0.9, 0.95]
    #for l in lst:
     #   quintile_col = np.quantile(train,l, axis=1)
      #  new_train=np.column_stack((new_train, med_col))
    
    total_features= new_train.shape[0]
    
#     new_train_2 = new_train**2
#     new_train_3 = new_train**2
    
#     new_train=np.column_stack((new_train, new_train_2))
#     new_train=np.column_stack((new_train, new_train_3))
        
    return new_train
#Append few extra features/columns
def append_bin_features(train):
    
    min_col = np.min(train, axis=1).reshape((-1,1))
    max_col = np.max(train, axis=1).reshape((-1,1))
    mean_col = np.mean(train, axis=1).reshape((-1,1))
    std_col = np.std(train, axis=1).reshape((-1,1))
    sum_col = np.sum(train, axis=1).reshape((-1,1))
    med_col = np.median(train, axis=1).reshape((-1,1))
    skew_col = scipy.stats.skew(train, axis=1).reshape((-1,1))
    kurtosis_col = scipy.stats.kurtosis(train, axis=1).reshape((-1,1))

    new_train = train.copy()
    
    new_train=np.column_stack((new_train, min_col))
    new_train=np.column_stack((new_train, max_col))
    new_train=np.column_stack((new_train, mean_col))
    new_train=np.column_stack((new_train, std_col))
    new_train=np.column_stack((new_train, sum_col))
    new_train=np.column_stack((new_train, med_col))
    new_train=np.column_stack((new_train, skew_col))
    new_train=np.column_stack((new_train, kurtosis_col))
    
    
    
    bin_train = train.copy()
    scaler = MinMaxScaler()
    scaler.fit(bin_train)
    min_max_train = np.round(scaler.transform(bin_train) * 10.0)
    
    scaler2 = MinMaxScaler()
    scaler2.fit(min_max_train)
    min_max_train = scaler2.transform(min_max_train)
    
    new_train=np.column_stack((new_train, min_max_train))
    
    return new_train

def classify(X_train, X_test, y_train, y_test, nest = 500, max_depth=3):
#     scale_pos_weight = float(np.sum(y_train == 1))/y_train.shape[0]
    scale_pos_weight = 10
    print("Nest",nest)
    classifier = XGBClassifier(random_state=42,
                      scale_pos_weight=scale_pos_weight,
                      max_depth=max_depth,
                      nestimator=nest, 
                      learning_rate=0.1,  
                      colsample_bytree = 0.4,
                      subsample = 0.8,
                      objective='binary:logistic')
    eval_set = [(X_train, y_train), (X_test, y_test)]
    
    eval_metric = ["auc"]
    classifier.fit(X_train, y_train, eval_metric=eval_metric, eval_set=eval_set,verbose= False)
    
    predictions = classifier.predict(X_train)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, predictions)
    roc_auc_tr = auc(false_positive_rate, true_positive_rate)
    pred_test = classifier.predict(X_test)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, pred_test)
    roc_auc_test = auc(false_positive_rate, true_positive_rate)
    return roc_auc_tr, roc_auc_test, classifier



In [12]:
org_train, org_labels, org_test,ids = read_input()

Train Shape (200000, 200)
Test Shape (200000, 200)


In [16]:
tr_auc,test_auc,classifier= classify(org_train, org_train, org_labels, org_labels)
print("Training and Test AUC:", tr_auc, test_auc)

test_predictions = classifier.predict(org_test)
columns=['ID_code','target']
query_output = pd.DataFrame(columns=columns)
query_output['ID_code']= ids
query_output['target']=test_predictions
query_output.to_csv('output_xgboost.csv', index=False,sep=',')   
print(query_output['target'],query_output['ID_code'])

Nest 25
Training and Test AUC: 0.7731611742299974 0.7731611742299974


PermissionError: [Errno 13] Permission denied: 'output_xgboost.csv'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.20)
n_estimators = [1, 16, 32, 64, 100,500]
train_results=[]
test_results=[]
for nest in n_estimators:
    tr_auc,test_auc,classifier = classify(X_train, X_test, y_train, y_test, nest=nest)
    train_results.append(tr_auc)
    test_results.append(test_auc)
    print("Number of Trees", nest, tr_auc, test_auc)


line1, = plt.plot(n_estimators, train_results, 'b', label='Train AUC')
line2, = plt.plot(n_estimators, test_results, 'r', label='Validation AUC')
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel('AUC score')
plt.xlabel('n_estimators')
plt.show()
 

In [6]:
pwd


'C:\\Users\\thm_m\\Desktop\\ML\\ML'

In [8]:
import os
os.chdir(
'C:\\Users\\thm_m\\Desktop\\ML')

In [11]:
pwd

'C:\\Users\\thm_m\\Desktop\\ML'